In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn import metrics as m
from thundersvm import SVC as svmgpu
import calculateWeightUsingGa2 as aresult
import pandas as pd
import itertools
import random
from sklearn.ensemble import VotingClassifier

randomseed = 42
np.random.seed(randomseed)

<IPython.core.display.Javascript object>

1. Read Dataset 
===

In [3]:
xtest = np.array(pd.read_csv("../dataset/xtest.txt"))
xtrain = np.array(pd.read_csv("../dataset/xtrain.txt"))
ytest_original = np.array(pd.read_csv("../dataset/ytest.txt")).ravel()
ytrain_original = np.array(pd.read_csv("../dataset/ytrain.txt")).ravel()

ytrain = ytrain_original.copy()
ytest = ytest_original.copy()

<IPython.core.display.Javascript object>

In [4]:
# member values
clf = []
acc = []
finalacc = []
ypredproba_all = []
ypredconfprob_all = []

<IPython.core.display.Javascript object>

In [5]:
# orginal score using random forest classifier
rf = RandomForestClassifier(random_state=randomseed, n_estimators=10)
rf.fit(xtrain, ytrain)
print("original score", m.f1_score(ytest, rf.predict(xtest), average="weighted"))

original score 0.6863966818349385


<IPython.core.display.Javascript object>

In [6]:
def swapcolumns(trainval, testval, coldindexval):
    trainval[trainval != coldindexval] = 5
    testval[testval != coldindexval] = 5

    trainval[trainval == coldindexval] = 0
    trainval[trainval == 5] = 1

    testval[testval == coldindexval] = 0
    testval[testval == 5] = 1

    return trainval, testval


<IPython.core.display.Javascript object>

2. Generate baseline classifiers 
===

In [7]:
#================================================= 
# Class 0
# ===========================
ytrain = ytrain_original.copy()
ytest = ytest_original.copy()
ytrain,ytest= swapcolumns(ytrain,ytest,0)
#=================================================

rf=RandomForestClassifier(random_state=randomseed, n_estimators=100)
rf.fit(xtrain,ytrain)
rfpred=rf.predict(xtest)
print(m.f1_score(ytest,rfpred,average='weighted')) 

clf.append(rf)
acc.append(m.f1_score(ytest,rfpred,average='weighted'))
ypredproba_all.append(rf.predict_proba(xtest)) 

confmat=m.confusion_matrix(ytest,rfpred)
confsumh=np.sum(confmat,axis=0)
propconfmat=confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:,i]= 100*propconfmat[:,i]/confsumh[i] 
ypredconfprob_all.append(propconfmat/100)


# =================================================
# classs 1
# =================================================
ytrain = ytrain_original.copy()
ytest = ytest_original.copy()
ytrain, ytest = swapcolumns(ytrain, ytest, 1)
# =================================================

rf = RandomForestClassifier(random_state=randomseed, n_estimators=100)
rf.fit(xtrain, ytrain)
rfpred = rf.predict(xtest)
print(m.f1_score(ytest, rfpred,average='weighted'))

clf.append(rf)
acc.append(m.f1_score(ytest, rfpred,average='weighted'))
ypredproba_all.append(rf.predict_proba(xtest))

confmat = m.confusion_matrix(ytest, rfpred)
confsumh = np.sum(confmat, axis=0)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
ypredconfprob_all.append(propconfmat / 100) 

#=================================================
# classs 2
#=================================================

ytrain=ytrain_original.copy()
ytest=ytest_original.copy()
ytrain,ytest= swapcolumns(ytrain,ytest,2)

#=================================================

rf=RandomForestClassifier(random_state=randomseed, n_estimators=100)
rf.fit(xtrain,ytrain)
rfpred=rf.predict(xtest)
print(m.f1_score(ytest,rfpred,average='weighted'))

clf.append(rf)
acc.append(m.f1_score(ytest,rfpred,average='weighted'))
ypredproba_all.append(rf.predict_proba(xtest))

confmat=m.confusion_matrix(ytest,rfpred)
confsumh=np.sum(confmat,axis=0)
propconfmat=confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:,i]= 100*propconfmat[:,i]/confsumh[i] 
ypredconfprob_all.append(propconfmat/100)

# #=================================================



0.7982938132290937
0.7288132220617086
0.8848939367275385


<IPython.core.display.Javascript object>

In [8]:

import calculateWeightUsingGa2 as aresult
weightvalga=aresult.getbestvalues(acc)

finalcol = np.zeros((ytest.shape[0], 3))
finalcol[:, 0] = ypredproba_all[0][:, 0]*weightvalga[0]
finalcol[:, 1] = ypredproba_all[1][:, 0]*weightvalga[1]
finalcol[:, 2] = ypredproba_all[2][:, 0]*weightvalga[2]

finalpred = np.argmax(finalcol, axis=1)

ytest = ytest_original.copy()
print('trial1_scores f1_score',m.f1_score(ytest, finalpred,average='weighted')) 

# #=================================================

import calculateWeightUsingGa2 as aresult
weightvalga=aresult.getbestvalues([
    ypredconfprob_all[0][0][0] * ypredconfprob_all[1][1][1]*ypredconfprob_all[2][1][1],
    ypredconfprob_all[1][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[2][1][1],
    ypredconfprob_all[2][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[1][1][1]])

finalcol = np.zeros((ytest.shape[0], 3))
finalcol[:, 0] = ypredproba_all[0][:, 0]*weightvalga[0]
finalcol[:, 1] = ypredproba_all[1][:, 0]*weightvalga[1]
finalcol[:, 2] = ypredproba_all[2][:, 0]*weightvalga[2]

finalpred = np.argmax(finalcol, axis=1)

ytest = ytest_original.copy()
print('trial2_scores f1_score',m.f1_score(ytest, finalpred,average='weighted')) 

# #=================================================

import calculateWeightUsingGa2 as aresult
weightvalga=aresult.getbestvalues([
    ypredconfprob_all[0][0][0] * ypredconfprob_all[1][1][1]*ypredconfprob_all[2][1][1],
    ypredconfprob_all[1][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[2][1][1],
    ypredconfprob_all[2][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[1][1][1]])

finalcol = np.zeros((ytest.shape[0], 3))
finalcol[:, 0] = ypredproba_all[0][:, 0] * weightvalga[0] + ypredproba_all[0][:, 1] * ypredconfprob_all[0][0][1]
finalcol[:, 1] = ypredproba_all[1][:, 0] * weightvalga[1] + ypredproba_all[1][:, 1] * ypredconfprob_all[1][0][1]
finalcol[:, 2] = ypredproba_all[2][:, 0] * weightvalga[2] + ypredproba_all[2][:, 1] * ypredconfprob_all[2][0][1]

finalpred = np.argmax(finalcol, axis=1)

ytest = ytest_original.copy()
print('trial3_scores f1_score',m.f1_score(ytest, finalpred,average='weighted'))


# #=================================================

import calculateWeightUsingGa2 as aresult
weightvalga=aresult.getbestvalues([
    ypredconfprob_all[0][0][0] * ypredconfprob_all[1][1][1]*ypredconfprob_all[2][1][1] ,
    -ypredconfprob_all[0][0][1],

    ypredconfprob_all[1][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[2][1][1],
    -ypredconfprob_all[1][0][1],

    ypredconfprob_all[2][0][0] * ypredconfprob_all[0][1][1]*ypredconfprob_all[1][1][1],
    -ypredconfprob_all[2][0][1],

    ])

finalcol = np.zeros((ytest.shape[0], 3))
finalcol[:, 0] = ypredproba_all[0][:, 0]*weightvalga[0] + ypredproba_all[0][:, 1]*weightvalga[1]
finalcol[:, 1] = ypredproba_all[1][:, 0]*weightvalga[2] + ypredproba_all[1][:, 1]*weightvalga[3]
finalcol[:, 2] = ypredproba_all[2][:, 0]*weightvalga[4] + ypredproba_all[2][:, 1]*weightvalga[5]

finalpred = np.argmax(finalcol, axis=1)

ytest = ytest_original.copy()
print('trial4_scores f1_score',m.f1_score(ytest, finalpred,average='weighted'))


# #=================================================

import calculateWeightUsingGa2 as aresult
weightvalga=aresult.getbestvalues([
    ypredconfprob_all[0][0][0] ,
    ypredconfprob_all[1][0][0] ,
    ypredconfprob_all[2][0][0] ])

finalcol = np.zeros((ytest.shape[0], 3))
finalcol[:, 0] = weightvalga[0] * (ypredproba_all[0][:, 0] * ypredconfprob_all[0][0][0] + ypredproba_all[0][:, 1] * ypredconfprob_all[0][0][1])
finalcol[:, 1] = weightvalga[1] * (ypredproba_all[1][:, 0] * ypredconfprob_all[1][0][0] + ypredproba_all[1][:, 1] * ypredconfprob_all[1][0][1])
finalcol[:, 2] = weightvalga[2] * (ypredproba_all[2][:, 0] * ypredconfprob_all[2][0][0] + ypredproba_all[2][:, 1] * ypredconfprob_all[2][0][1])

finalpred = np.argmax(finalcol, axis=1)

ytest = ytest_original.copy()
print('trial5_scores f1_score',m.f1_score(ytest, finalpred,average='weighted')) 

# #=================================================

import calculateWeightUsingGa2 as aresult
weightvalga=aresult.getbestvalues([
    ypredconfprob_all[0][0][0] ,
    ypredconfprob_all[1][0][0] ,
    ypredconfprob_all[2][0][0] ])

finalcol = np.zeros((ytest.shape[0], 3))
finalcol[:, 0] =  (weightvalga[0] *ypredproba_all[0][:, 0] * ypredconfprob_all[0][0][0] + ypredproba_all[0][:, 1] * ypredconfprob_all[0][0][1])
finalcol[:, 1] = (weightvalga[1] * ypredproba_all[1][:, 0] * ypredconfprob_all[1][0][0] + ypredproba_all[1][:, 1] * ypredconfprob_all[1][0][1])
finalcol[:, 2] =  (weightvalga[2] *ypredproba_all[2][:, 0] * ypredconfprob_all[2][0][0] + ypredproba_all[2][:, 1] * ypredconfprob_all[2][0][1])

finalpred = np.argmax(finalcol, axis=1)

ytest = ytest_original.copy()
print('trial6_scores f1_score',m.f1_score(ytest, finalpred,average='weighted'))


0
1
2
3
4
5
6
7
8
9
2.3630176337806312 [0.98567296 0.95740995 0.99486692]
trial1_scores f1_score 0.7058590982763423
0
1
2
3
4
5
6
7
8
9
1.4900723001710645 [0.92717009 0.90035309 0.99628568]
trial2_scores f1_score 0.7102797006976361
0
1
2
3
4
5
6
7
8
9
1.5085279209357518 [0.96754017 0.90615642 0.99427627]
trial3_scores f1_score 0.7040004748338081
0
1
2
3
4
5
6
7
8
9
0.9880227567436647 [0.96346402 0.79758651 0.86317886 0.75064028 0.97412023 0.85314707]
trial4_scores f1_score 0.4890451478774598
0
1
2
3
4
5
6
7
8
9
2.1128398011375147 [0.98749441 0.88706228 0.99659541]
trial5_scores f1_score 0.7193852813852815
0
1
2
3
4
5
6
7
8
9
2.117950508219453 [0.9848773  0.89504325 0.99783267]
trial6_scores f1_score 0.7181962341710466


<IPython.core.display.Javascript object>